# TI3145TU Midterm Assignment 
## Football Players Wages

We hope you enjoy this assignment, good luck!

Student names: XXX

Student numbers: XXX

### Imports

In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor

import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

### Data Loading and Preprocessing

Data is read from the csv file 'football_wages.csv'.

To convert the nationaility_name column from strings to numeric data, each nationality is assigned an integer, based on its ranking of average log_wages.

Since the dataset is significantly smaller than the "football_autograder.csv", all the data is kept from "football_wages.csv" for training and validation. Overfitting is minimized by means of cross validation. Then the error estimation is done by averaging the predictions of the models over the entire Y set, which they have been tunned for different data points. 

In [18]:
data = pd.read_csv('football_wages.csv')
data_autograder = pd.read_csv('football_autograder.csv')



sort_nationalities = True

if sort_nationalities == True:
    # sort the data in ascending order of average log_wages by nationality
    
    mean_by_nationality = data.groupby('nationality_name')['log_wages'].mean().reset_index()

    mean_by_nationality_sorted = mean_by_nationality.sort_values(by='log_wages', ascending=False)

    sorted_data = pd.merge(data, mean_by_nationality_sorted, on='nationality_name', suffixes=('', '_mean'))

    sorted_data = sorted_data.sort_values(by='log_wages_mean', ascending=False).drop(columns='log_wages_mean')

    # convert the nationality strings to integers, with 0 being the country with lowest average log_wages
    
    sorted_data['nationality_name'], uniques = pd.factorize(data['nationality_name'])

    X = sorted_data.iloc[:, :-1]

else:
    data.drop('nationality_name',axis=1,inplace=True)
    X = data.iloc[:, :-1]


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Y = sorted_data.iloc[:, -1]
X_train, X_val, y_train, y_val = train_test_split(X_scaled, Y, test_size=0.3, random_state=100)




Mean of features by nationality (divided by log_wages):             nationality_name        age  height_cm  weight_kg    overall  \
0                b'Albania'   7.550218  53.140989  22.361188  18.911727   
1                b'Algeria'   6.650764  44.375501  18.306284  17.572253   
2                 b'Angola'   6.874100  47.325537  20.093524  17.802157   
3    b'Antigua and Barbuda'   8.482201  52.407885  23.326053  18.782017   
4              b'Argentina'   7.383422  49.468074  20.749189  18.933768   
..                      ...        ...        ...        ...        ...   
126            b'Venezuela'   8.494369  60.433273  24.836869  21.359174   
127              b'Vietnam'   8.179265  56.951921  26.658346  19.387888   
128                b'Wales'   7.011815  50.467049  20.969915  17.857193   
129               b'Zambia'  10.003816  68.915179  29.640937  24.083261   
130             b'Zimbabwe'   6.861159  50.338503  21.003548  18.553134   

     potential  attacking_crossing  attac

In [19]:
sorted_data.head()

,age,height_cm,weight_kg,nationality_name,overall,potential,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,...,movement_agility_mean,movement_reactions_mean,movement_balance_mean,defending_standing_tackle_mean,defending_sliding_tackle_mean,goalkeeping_diving_mean,goalkeeping_handling_mean,goalkeeping_kicking_mean,goalkeeping_positioning_mean,goalkeeping_reflexes_mean
2999,28.0,183.0,80.0,0,74.0,74.0,69.0,69.0,75.0,76.0,...,16.186578,15.539115,14.891652,6.042989,4.963884,2.589853,3.021495,1.942389,1.510747,1.942389
860,25.0,186.0,82.0,1,70.0,75.0,61.0,69.0,69.0,64.0,...,17.837483,16.715628,17.613112,7.516424,8.413907,1.121854,2.243709,2.355894,1.570596,1.346225
4564,32.0,177.0,75.0,0,83.0,83.0,77.0,80.0,68.0,83.0,...,17.837483,16.715628,17.613112,7.516424,8.413907,1.121854,2.243709,2.355894,1.570596,1.346225
2511,28.0,172.0,76.0,2,71.0,71.0,64.0,63.0,50.0,65.0,...,18.570622,14.902351,19.487690,8.712144,3.897538,3.209737,2.980470,2.292669,2.521936,2.521936
3330,31.0,170.0,69.0,3,70.0,70.0,60.0,72.0,68.0,64.0,...,18.135191,16.507674,18.135191,9.997605,8.137586,2.325024,1.395015,3.022532,2.557527,1.627517


### Define ftraining functions

In [20]:
def train_and_test_knn(k, x_train, y_train, x_val,y_val):
    # Initialize the KNN regressor
    model = KNeighborsRegressor(n_neighbors=k)

    # Train the KNN regressor
    model.fit(x_train, y_train)

    # Predict on the validation set
    y_pred = model.predict(x_val)

    # Calculate error
    error = np.mean(np.abs(y_val - y_pred))

    return model,error

def tune_and_store_knn(x_train,y_train,x_val,y_val):
    min_error = np.inf
    for k in range(1,40):
        model,error = train_and_test_knn(k,x_train,y_train,x_val,y_val)
        #print('K: ',k,' Error: ',error)

        if error < min_error:
            min_error = error
            best_k = k
            best_model = model
    print("Best K:", best_k)
    return best_k,best_model,min_error

def train_and_test_SGD(parameters,x_train,y_train,x_val,y_val): # To be implemented by Edlyn
    return
def tune_and_store_SGD(x_train,y_train,x_val,y_val): # To be implemented by Edlyn
    return

### Train and store tunned KNNs

In [21]:
#########################################
#--------KNN Cross validation Training 
#########################################
number_of_models = 15
best_KNN_model_list =[]
best_k_list = []
val_error_list = []

# Perform cross validation
for i in range(number_of_models):
    print("-----\nModel: ", i)
    x_train, x_val, y_train, y_val = train_test_split(X,Y,test_size=0.2,random_state=i)
    best_k,best_model,min_error = tune_and_store_knn(x_train,y_train,x_val,y_val)
    best_KNN_model_list.append(best_model)
    val_error_list.append(min_error)
    best_k_list.append(best_k)

print('Best K list: ',best_k_list)
norm_val_error = np.mean(np.array(val_error_list))
print('Normalized MAE on validation sets: ',norm_val_error)


-----
Model:  0
Best K: 30
-----
Model:  1
Best K: 17
-----
Model:  2
Best K: 13
-----
Model:  3
Best K: 16
-----
Model:  4
Best K: 28
-----
Model:  5
Best K: 28
-----
Model:  6
Best K: 15
-----
Model:  7
Best K: 27
-----
Model:  8
Best K: 19
-----
Model:  9
Best K: 34
-----
Model:  10
Best K: 14
-----
Model:  11
Best K: 22
-----
Model:  12
Best K: 19
-----
Model:  13
Best K: 30
-----
Model:  14
Best K: 8
Best K list:  [30, 17, 13, 16, 28, 28, 15, 27, 19, 34, 14, 22, 19, 30, 8]
Normalized MAE on validation sets:  0.6381739715635771


### Train and store best SGD

In [22]:
#Edlyn should implement this


ValueError: Found input variables with inconsistent numbers of samples: [3500, 4000]

In [22]:
# Instantiate the SGDRegressor
sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)

# Define the parameter grid for alpha (regularization strength) and learning rate
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Tuning regularization strength
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],  # Different learning rate strategies
    'eta0': [0.001, 0.01, 0.1, 1]  # Initial learning rate for strategies that require it
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(sgd_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_scaled, y_train)

# Get the best model from grid search
best_sgd = grid_search.best_estimator_

# Predict on the test set with the best model
y_pred = best_sgd.predict(X_test_scaled)

# Evaluate the model using MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"Best MAE: {mae}")
print(f"Best Parameters: {grid_search.best_params_}")

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stocha

Best MAE: 0.27692806009280757
Best Parameters: {'alpha': 0.0001, 'eta0': 0.1, 'learning_rate': 'adaptive'}


In [ ]:
# TODO Replace this with your own estimate of the MAE of your best model
estimate_MAE_on_new_data = np.array([1.0])

# TODO Replace this with the predictions of your best model
# via e.g. prediction = model.predict(data_autograder)
# your predictions here should again be the $log_{10}(wage)$ of the football player, just as in the provided data. 
predictions_autograder_data = np.array([-1] * 14178)

# Upload this file to the Vocareum autograder:
result = np.append(estimate_MAE_on_new_data, predictions_autograder_data)
pd.DataFrame(result).to_csv("autograder_submission.txt", index=False, header=False)